In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable("green")

C:\Users\crab\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyspark\sql\dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [4]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour,  
    PULocationID AS  zone,

    -- Revenue calculation  
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) AS number_records
 
FROM
    green
WHERE
    lpep_pickup_datetime >='2020-01-01 00:00:00'
GROUP BY
    1, 2 
ORDER BY
    1,2
""")

In [5]:
df_green_revenue.show()

+-------------------+----+----------------------------+--------------+
|               hour|zone|revenue_monthly_total_amount|number_records|
+-------------------+----+----------------------------+--------------+
|2020-01-01 00:00:00|   7|           769.7299999999999|            45|
|2020-01-01 00:00:00|  17|                      195.03|             9|
|2020-01-01 00:00:00|  18|                         7.8|             1|
|2020-01-01 00:00:00|  22|                        15.8|             1|
|2020-01-01 00:00:00|  24|                        87.6|             3|
|2020-01-01 00:00:00|  25|                       531.0|            26|
|2020-01-01 00:00:00|  29|                        61.3|             1|
|2020-01-01 00:00:00|  32|           68.94999999999999|             2|
|2020-01-01 00:00:00|  33|                      317.27|            11|
|2020-01-01 00:00:00|  35|                      129.96|             5|
|2020-01-01 00:00:00|  36|                      295.34|            11|
|2020-

In [17]:
df_green_revenue\
    .repartition(10)\
    .write.parquet('data/report/revenue/green',mode='overwrite')

In [14]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable("yellow")

In [15]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour,  
    PULocationID AS  zone,

    -- Revenue calculation  
    SUM(total_amount) AS revenue_monthly_total_amount,
    COUNT(1) AS number_records
 
FROM
    yellow
WHERE
    tpep_pickup_datetime >='2020-01-01 00:00:00'
GROUP BY
    1, 2 
ORDER BY
    1,2
""")

In [18]:
df_yellow_revenue\
    .repartition(10)\
    .write.parquet('data/report/revenue/yellow',mode='overwrite')

In [29]:
df_green_revenue_tmp = df_green_revenue\
                        .withColumnRenamed('revenue_monthly_total_amount','green_amount')\
                        .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_green_revenue\
                        .withColumnRenamed('revenue_monthly_total_amount','yellow_amount')\
                        .withColumnRenamed('number_records', 'yellow_number_records')

In [30]:
# null if there is missing value
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour','zone'], how ='outer') 

In [31]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  65|199.48999999999998|                  10|199.48999999999998|                   10|
|2020-01-01 00:00:00| 106|             10.56|                   1|             10.56|                    1|
|2020-01-01 01:00:00|  83|             87.34|                   8|             87.34|                    8|
|2020-01-01 01:00:00| 126|              15.8|                   1|              15.8|                    1|
|2020-01-01 01:00:00| 134|             34.86|                   4|             34.86|                    4|
|2020-01-01 02:00:00|  21|             43.95|                   2|             43.95|                    2|
|2020-01-01 02:00:00|  35|  

In [32]:
df_join.write.parquet('data/report/revenue/total')

In [33]:
df_join = spark.read.parquet('data/report/revenue/total')

In [34]:
df_zones = spark.read.parquet('zones/')

In [35]:
df_zones.show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [37]:
df_results = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [40]:
df_results.drop('LocationID').show()

+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|  Borough|                Zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+---------+--------------------+------------+
|2020-01-01 00:00:00|  32| 68.94999999999999|                   2| 68.94999999999999|                    2|    Bronx|           Bronxdale|   Boro Zone|
|2020-01-01 00:00:00|  41|1363.9599999999991|                  84|1363.9599999999991|                   84|Manhattan|      Central Harlem|   Boro Zone|
|2020-01-01 00:00:00|  55|            129.29|                   4|            129.29|                    4| Brooklyn|        Coney Island|   Boro Zone|
|2020-01-01 00:00:00|  63|              51.9|                   2|              51.9|   

In [42]:
df_results.drop('LocationID','zone').write.parquet('tmp/revenue-zones')